In [7]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from keras.datasets import mnist
import random
import sys
sys.path.append('../')
from Core_Functions import neural_networks, generate_image_set

In [15]:
# Load data and split into training and test
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [16]:
# Split training set further into training and validation sets
sample_size = 40000
X_train, X_val, y_train, y_val = train_images[:sample_size], train_images[sample_size:sample_size*2],  train_labels[:sample_size], train_labels[sample_size:sample_size*2]

In [17]:
combo_train_X, combo_train_y = generate_image_set.get_image_combinations(40000, X_train, y_train)
combo_val_X, combo_val_y = generate_image_set.get_image_combinations(20000, X_val, y_val)

combo_test_X, combo_test_y = generate_image_set.get_image_combinations(10000, test_images, test_labels)

In [19]:
# Reshape the data so they can be fed into the network, one-hot-encode the labels

train_images = combo_train_X.reshape((combo_train_X.shape[0], 28, 56, 1))
train_images = train_images/ 255

val_images = combo_val_X.reshape((combo_val_X.shape[0], 28, 56, 1))
val_images = val_images/255

test_images = combo_test_X.reshape((combo_test_X.shape[0], 28, 56, 1))
test_images = test_images/255

train_labels = to_categorical(combo_train_y)
val_labels = to_categorical(combo_val_y)

test_labels = to_categorical(combo_test_y)

In [12]:
model = neural_networks.build_CNN((28, 56, 1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 54, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 27, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 12, 64)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 10, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [ ]:
# Perform k-fold cross-validation to understand the optimal number of epochs to train on
k = 4
num_val_samples = len(train_images)//k

num_epochs = 50

all_val_mae_histories = []
all_train_mae_histories = []

all_val_acc_histories = []
all_train_acc_histories = []

for i in range(k):
    
    print('processing fold #', i)
    val_data = train_images[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_labels[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
    [train_images[:i * num_val_samples],
    train_images[(i+1) * num_val_samples:]],
    axis = 0)
    
    partial_train_targets = np.concatenate(
    [train_labels[:i*num_val_samples],
    train_labels[(i+1)*num_val_samples:]],
    axis = 0)
    
    model = neural_networks.build_CNN((28, 56, 1))
    history = model.fit(partial_train_data,
              partial_train_targets,
              validation_data = (val_data, val_targets),
              epochs = num_epochs,
              batch_size = 64,
              verbose = 0)
    
    val_mae_history = history.history['val_mae']
    all_val_mae_histories.append(val_mae_history)
    
    train_mae_history = history.history['mae']
    all_train_mae_histories.append(train_mae_history)
    
    val_acc_history = history.history['val_accuracy']
    all_val_acc_histories.append(val_acc_history)
    
    train_acc_history = history.history['accuracy']
    all_train_acc_histories.append(train_acc_history)

In [ ]:
# Compute the average of the per-epoch MAE scores for all folds
# Building the history of successive mean K-fold validation scores

average_val_mae_history = [
    np.mean([x[i] for x in all_val_mae_histories]) for i in range(num_epochs)]

average_train_mae_history = [
    np.mean([x[i] for x in all_train_mae_histories]) for i in range(num_epochs)]

plt.plot(range(1, len(average_train_mae_history) + 1), average_train_mae_history, label = 'Training', c ='mediumseagreen', lw=2.5)
plt.plot(range(1, len(average_val_mae_history) + 1), average_val_mae_history, label='Validation', c = 'mediumvioletred', lw=2.5)

plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.tight_layout()
plt.title('CNN Mean Absolute Error')
plt.savefig('Q1_Figures/Q1_CNN_MAE.png', dpi = 400, bbox_inches="tight")

In [ ]:
# Compute the average of the per-epoch Accuracy scores for all folds
# Building the history of successive mean K-fold validation scores

average_val_acc_history = [
    np.mean([x[i] for x in all_val_acc_histories]) for i in range(num_epochs)]

average_train_acc_history = [
    np.mean([x[i] for x in all_train_acc_histories]) for i in range(num_epochs)]

plt.plot(range(1, len(average_train_acc_history) + 1), average_train_acc_history, label = 'Training', c ='mediumseagreen', lw=2.5)
plt.plot(range(1, len(average_val_acc_history) + 1), average_val_acc_history, label='Validation', c = 'mediumvioletred', lw=2.5)

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.title('CNN Accuracy')
plt.savefig('Q1_Figures/Q1_CNN_Accuracy.png', dpi = 400, bbox_inches="tight")

In [13]:
# Rebuild the model and train on full dataset (60k) rather than splitting it (48k)
# Do this for the optimal number of epochs
    # It seems like around 20 epochs the validation MAE and Accuracy flatten, so this will be used
model = neural_networks.build_CNN((28, 56, 1))
model.fit(train_images, train_labels, epochs = 20, batch_size = 64)

Epoch 1/20
625/625 [==============================] - 29s 46ms/step - loss: 0.4584 - accuracy: 0.8522 - mae: 0.0436
Epoch 2/20
625/625 [==============================] - 43s 69ms/step - loss: 0.1289 - accuracy: 0.9607 - mae: 0.0127
Epoch 3/20
625/625 [==============================] - 35s 56ms/step - loss: 0.0716 - accuracy: 0.9779 - mae: 0.0071
Epoch 4/20
625/625 [==============================] - 35s 56ms/step - loss: 0.0457 - accuracy: 0.9859 - mae: 0.0045
Epoch 5/20
625/625 [==============================] - 35s 56ms/step - loss: 0.0303 - accuracy: 0.9904 - mae: 0.0030
Epoch 6/20
625/625 [==============================] - 36s 57ms/step - loss: 0.0219 - accuracy: 0.9932 - mae: 0.0020
Epoch 7/20
625/625 [==============================] - 35s 56ms/step - loss: 0.0147 - accuracy: 0.9957 - mae: 0.0014
Epoch 8/20
625/625 [==============================] - 35s 56ms/step - loss: 0.0124 - accuracy: 0.9964 - mae: 0.0011
Epoch 9/20
625/625 [==============================] - 35s 56ms/step - lo

In [20]:
test = model.evaluate(test_images, test_labels)
test

313/313 [==============================] - 1s 4ms/step - loss: 0.1499 - accuracy: 0.9838 - mae: 0.0033


[0.1498870849609375, 0.9837999939918518, 0.003317464841529727]

In [ ]:
# Verify above by visually inspecting images from the predicted set
num_val_images = 10000

# Run this cell repeatedly to see multiple examples
check_index = random.randint(0,num_val_images)
plt.imshow(val_images[check_index].reshape(28,56), cmap = plt.cm.binary)
print("Predicted: " + str(np.argmax(y_predict[check_index])))
print("True: " + str(np.argmax(val_labels[check_index])))
#plt.savefig('Q1_Figures/Q1_Example_Image_' + str(check_index) + '.png', dpi = 400, bbox_inches="tight")

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("Q1_Trained_Models/CNN_Model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Q1_Trained_Models/CNN_Model.h5")
print("Saved model to disk")